<a href="https://colab.research.google.com/github/faizer1989/AI_recipes/blob/main/Product_Performance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
#@title Import python libraries
import sys
!pip install -Uqq pandas-profiling[notebook]
!pip install -Uqq pycaret
!jupyter nbextension enable --py widgetsnbextension

# Standard Library Imports
from pathlib import Path

# Installed packages
import pandas as pd
from ipywidgets import widgets

# Our package
from pandas_profiling import ProfileReport
from pandas_profiling.utils.cache import cache_file

#Enable pycaret on Colab
from pycaret.utils import enable_colab
enable_colab()
%matplotlib inline

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Colab mode enabled.


## Collecting Labeled Data

**Download the dataset here:** https://www.kaggle.com/blastchar/telco-customer-churn

Upload the csv file to Files (the menu option on the left side of the screen)

In [37]:
df = pd.read_excel("/content/3.3 BricktopBullybits public.xlsx")

In [38]:
df.head()

,FURNITURE CATEGORY,PRODUCT CATEGORY,COUNTRY,VENDOR,DELIVERY METHOD,SKU,DESCRIPTION,POWER,KD,PRIMARY MATERIAL 1,PRIMARY MATERIAL 2,PRIMARY COLOR 1,PRIMARY COLOR 2,PAINTED METAL,EXRTs,DELIVERIES,EXRT%,EXRT%>6%
0,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310037,ARMLESS CHAIR,NO,NO,FABRIC,NaN,WHITE,NaN,NO,855,7716,0.110809,True
1,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310042,ARMLESS CHAIR,NO,NO,FABRIC,NaN,GRAY,NaN,NO,431,7617,0.056584,False
2,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310040,ARMREST,NO,NO,FABRIC,NaN,WHITE,NaN,NO,219,2513,0.087147,True
3,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310045,ARMREST,NO,NO,FABRIC,NaN,GRAY,NaN,NO,151,2443,0.061809,True
4,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310043,WEDGE,NO,NO,FABRIC,NaN,GRAY,NaN,NO,106,1805,0.058726,False


In [48]:
df = df.drop(columns = ["EXRTs", "EXRT%"])

In [49]:
df.head()

,FURNITURE CATEGORY,PRODUCT CATEGORY,COUNTRY,VENDOR,DELIVERY METHOD,SKU,DESCRIPTION,POWER,KD,PRIMARY MATERIAL 1,PRIMARY MATERIAL 2,PRIMARY COLOR 1,PRIMARY COLOR 2,PAINTED METAL,DELIVERIES,EXRT%>6%
0,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310037,ARMLESS CHAIR,NO,NO,FABRIC,NaN,WHITE,NaN,NO,7716,True
1,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310042,ARMLESS CHAIR,NO,NO,FABRIC,NaN,GRAY,NaN,NO,7617,False
2,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310040,ARMREST,NO,NO,FABRIC,NaN,WHITE,NaN,NO,2513,True
3,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310045,ARMREST,NO,NO,FABRIC,NaN,GRAY,NaN,NO,2443,True
4,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310043,WEDGE,NO,NO,FABRIC,NaN,GRAY,NaN,NO,1805,False


##Analyzing Your Data

In [50]:
profile = ProfileReport(df, title="Product Performance")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Data Preparation

In [51]:
data = df.sample(frac=0.95, random_state=786)
data_unseen = df.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (94, 16)
Unseen Data For Predictions: (5, 16)


In [52]:
data.head()

,FURNITURE CATEGORY,PRODUCT CATEGORY,COUNTRY,VENDOR,DELIVERY METHOD,SKU,DESCRIPTION,POWER,KD,PRIMARY MATERIAL 1,PRIMARY MATERIAL 2,PRIMARY COLOR 1,PRIMARY COLOR 2,PAINTED METAL,DELIVERIES,EXRT%>6%
0,LIVING ROOM,SECTIONAL,CHIMERICA,GINM,TRUCK,G1310041,BLUETOOTH STRG CNSL,YES,NO,FABRIC,PLASTIC,WHITE,NaN,NO,704,True
1,LIVING ROOM,DEEP SEATING,CHIMERICA,OUZY,TRUCK,O1500163,DUAL PWR RECL CONSOLE LOVESEAT,YES,YES,LEATHER,NaN,BROWN,NaN,NO,407,False
2,LIVING ROOM,DEEP SEATING,CHIMERICA,YENS,TRUCK,Y1034177,DUAL PWR RECL CONSOLE LOVESEAT,YES,YES,LEATHER,NaN,GRAY,NaN,NO,251,True
3,LIVING ROOM,SECTIONAL,CHIMERICA,YENS,TRUCK,Y1034105,DUAL PWR RAF RECLINER,YES,YES,LEATHER,PU,WHITE,GRAY,NO,587,False
4,LIVING ROOM,DEEP SEATING,CHIMERICA,OUZY,TRUCK,O1500181,STATIONARY LOVESEAT,NO,YES,LEATHER,NaN,DARK BROWN,NaN,NO,402,False


In [53]:
from pycaret.classification import *
data_prep = setup(data, target = 'EXRT%>6%')

,Description,Value
0,session_id,3206
1,Target,EXRT%>6%
2,Target Type,Binary
3,Label Encoded,"False: 0, True: 1"
4,Original Data,"(94, 16)"
5,Missing Values,True
6,Numeric Features,1
7,Categorical Features,14
8,Ordinal Features,False
9,High Cardinality Features,False


## Training the Model

In [54]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
nb,Naive Bayes,0.8143,0.8236,0.8750,0.8083,0.8110,0.6220,0.6664,0.019
dt,Decision Tree Classifier,0.8119,0.8083,0.7833,0.8767,0.8081,0.6195,0.6529,0.017
gbc,Gradient Boosting Classifier,0.8119,0.8292,0.7833,0.8633,0.8051,0.6194,0.6479,0.067
lr,Logistic Regression,0.7976,0.8056,0.8500,0.7983,0.8058,0.5915,0.6279,0.486
lda,Linear Discriminant Analysis,0.7833,0.8250,0.8750,0.7417,0.7914,0.5630,0.5966,0.022
lightgbm,Light Gradient Boosting Machine,0.7833,0.7722,0.6750,0.9083,0.7505,0.5720,0.6102,0.084
rf,Random Forest Classifier,0.7667,0.8847,0.8500,0.7433,0.7760,0.5274,0.5707,0.469
ada,Ada Boost Classifier,0.7667,0.7444,0.7583,0.8183,0.7562,0.5302,0.5706,0.094
et,Extra Trees Classifier,0.7548,0.8681,0.7833,0.7750,0.7490,0.5047,0.5457,0.450
ridge,Ridge Classifier,0.7024,0.0000,0.7583,0.7033,0.7126,0.4068,0.4369,0.017


In [55]:
nb = create_model('nb')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7143,0.7500,0.6667,0.6667,0.6667,0.4167,0.4167
1,0.8571,0.9167,1.0000,0.7500,0.8571,0.7200,0.7500
2,0.7143,0.7917,1.0000,0.6667,0.8000,0.3636,0.4714
3,0.8571,0.8333,0.7500,1.0000,0.8571,0.7200,0.7500
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,0.6667,0.8889,0.3333,1.0000,0.5000,0.3333,0.4472
6,0.8333,0.8889,1.0000,0.7500,0.8571,0.6667,0.7071
7,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071
8,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071
9,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071


In [56]:
tuned_nb = tune_model(nb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7143,0.7500,0.6667,0.6667,0.6667,0.4167,0.4167
1,0.8571,0.9167,1.0000,0.7500,0.8571,0.7200,0.7500
2,0.7143,0.7917,1.0000,0.6667,0.8000,0.3636,0.4714
3,0.8571,0.8333,0.7500,1.0000,0.8571,0.7200,0.7500
4,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
5,0.6667,0.8889,0.3333,1.0000,0.5000,0.3333,0.4472
6,0.8333,0.8889,1.0000,0.7500,0.8571,0.6667,0.7071
7,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071
8,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071
9,0.8333,0.7222,1.0000,0.7500,0.8571,0.6667,0.7071


## Model Evaluation

In [57]:
evaluate_model(tuned_nb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [59]:
final_nb = finalize_model(tuned_nb)

## Using the Model to Make Predictions

In [60]:
predict_model(final_nb);

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.8276,0.8737,0.9444,0.8095,0.8718,0.6133,0.6305


In [84]:
unseen_predictions = predict_model(final_nb, data=data_unseen)
unseen_predictions.head()

,FURNITURE CATEGORY,PRODUCT CATEGORY,COUNTRY,VENDOR,DELIVERY METHOD,SKU,DESCRIPTION,POWER,KD,PRIMARY MATERIAL 1,PRIMARY MATERIAL 2,PRIMARY COLOR 1,PRIMARY COLOR 2,PAINTED METAL,DELIVERIES,EXRT%>6%,Label,Score
0,LIVING ROOM,DEEP SEATING,CHIMERICA,OUZY,TRUCK,O1500178,DUAL PWR RECL SOFA,YES,YES,LEATHER,NaN,BROWN,NaN,NO,802,True,True,1.0
1,LIVING ROOM,DEEP SEATING,CHIMERICA,OUZY,TRUCK,O1500127,DUAL PWR RECL CONSOLE LOVESEAT,YES,YES,LEATHER,NaN,DARK GRAY,NaN,NO,457,True,True,1.0
2,LIVING ROOM,DEEP SEATING,CHIMERICA,YENS,TRUCK,Y1034143,DUAL PWR RECL SOFA,YES,YES,LEATHER,NaN,BROWN,NaN,NO,250,True,True,1.0
3,OUTDOOR,TABLE,BADLANDS,TICS,TRUCK,S0822965,PURIFIED RECT DN TABLE,NO,YES,ALUMINUM,HDPE,GRAY,NaN,NO,148,True,False,1.0
4,OUTDOOR,TABLE,BADLANDS,TICS,TRUCK,S0822975,SPARTAN BREW SQUARE TBL W/ TEAK TOP,NO,YES,ALUMINUM,TEAK,WHITE,NaN,NO,12,False,False,1.0


In [86]:
from pycaret.utils import check_metric
check_metric(unseen_predictions['EXRT%>6%'], unseen_predictions['Label']=='True', metric = 'Accuracy')

0.8

## Saving the Model

In [76]:
save_model(final_nb,'Final Prediction Model 03Mar2022')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='EXRT%>6%',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_str...
                 ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                 ('cluster_all', 'passthrough'),
                 ('dummy', Dummify(target='EXRT%>6%')),
                 ('fix_perfect', Remove_100(target='EXRT%>6%')),
                 ('clean_name